In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
import pandas as pd

In [77]:
def get_Heston_chf(x, u, v0, r, rho, ka, th, sig, T):
    """
    Compute E[e^{1j*u*x}]
    x: log(S)
    u: variable for the chf
    v0: v0
    r: risk-free rate
    rho: correlation
    ka: kappa
    th: theta
    sig: sigma
    """
    gam = np.sqrt(sig**2*u*(u+1j) + (ka-1j*rho*sig*u)**2)
    half_gam_T = gam*T/2.0
    coth = 1.0/np.tanh(half_gam_T)
    log_numerator = ka*th*T*(ka-1j*rho*sig*u)/sig**2 + 1j*u*(T*r+x) - (u*(u+1j)*v0)/(gam*coth + ka - 1j*rho*sig*u)
    numerator = np.exp(log_numerator)
    denominator = (np.cosh(half_gam_T) + (ka-1j*rho*sig*u)/gam*np.sinh(half_gam_T))**(2.0*ka*th/sig**2)
    return numerator/denominator

def get_damped_Heston_chf(x, v, v0, r, rho, ka, th, sig, T, alp=0.75):
    chf_inp = v - (alp+1.0)*1j
    numerator = np.exp(-r*T) * get_Heston_chf(x, chf_inp, v0, r, rho, ka, th, sig, T)
    denominator = alp**2 + alp - v**2 +1j*(2.0*alp +1.0)*v
    return numerator / denominator

def get_Heston_fft_call(k, x, v0, r, rho, kappa, theta, sigma, T, alp=0.75, N=2**12, dk=0.01):# 2048*5
    u_max = 20
    u_list = np.linspace(0,u_max,num=N-1)
    # be careful about scales of dk and dv
#    dk = np.abs(k)*0.01
    dv = 2.0*np.pi/(N*dk)  # N*dv = 2*pi/dk

#    dk = 2.0*np.pi/(N*dv) # nu nu*zeta = 2*pi/N
    
#    print(dv*(N-1))
#    print('dk ds ', dk, ' ',dv)
    k_list = np.array([x + dk*u_elem - N*dk/2.0 for u_elem in u_list])
    v_list = np.array(range(N-1)) * dv
#    print('maxv ', v_list[-1])
#    if np.isinf(np.sinh(v_list[-1]**2)) | np.isinf(np.cosh(v_list[-1]**2)) | np.isinf(np.exp(v_list[-1]**2)):
#        print('Infinite ',v_list[-1]**2)
#        raise Exception('inifite')

    x_list = [] # values in frequency domain
    for (j,vj) in enumerate(v_list): # j=0 to N-1
        if j==0:
            kroneker_delta = 1.0
        else:
            kroneker_delta = 0.0
        simpson_coeff = dv/3.0*(3.0 + (-1)**j - kroneker_delta)
        xj = np.exp(1j*(N/2.0*dk-x)*vj)*simpson_coeff*get_damped_Heston_chf(x, vj, v0, r, rho, kappa, theta, sigma, T, alp)
        x_list.append(xj)

    fft_prices = np.exp(-alp*k)/np.pi*fft(x_list).real
#     print(k_list)
#     print(fft_prices)
    # interpolate results
    fft_price = np.interp(np.exp(k), np.exp(k_list), fft_prices)
#    print(fft_price)
    return fft_price

In [78]:
df = pd.read_csv('ivol_surface.csv')
df['log_k'] = np.log(df['k'])
df

,tenor,k,price,ivol,rel_error,log_k
0,0.019178,0.877228,0.124088,0.556128,-1.734493e-10,-0.130988
1,0.019178,0.897165,0.103274,0.392131,3.189898e-11,-0.108516
2,0.019178,0.927070,0.075043,0.400563,-5.730807e-10,-0.075726
3,0.019178,0.937039,0.064995,0.353658,5.798162e-10,-0.065030
4,0.019178,0.947007,0.055225,0.318121,1.111488e-10,-0.054448
...,...,...,...,...,...,...
194,2.032877,1.216157,0.067188,0.242079,5.480717e-09,0.195696
195,2.032877,1.256031,0.056621,0.238186,8.349152e-11,0.227957
196,2.032877,1.355716,0.037561,0.233383,6.431517e-09,0.304330
197,2.032877,1.475338,0.028829,0.247046,-5.815883e-09,0.388887


# Fitting Price Surface

In [79]:
# [theta = 0.132328, kappa = 10.980797; sigma = 1.7; rho = -0.351560; v0 = 0.065690]
params = [0.132328,10.980797,1.7,-0.351560,0.065690] # initial parameters
r = 0.0

In [80]:
df_surf = df[['tenor','log_k','price']].copy()

In [87]:
fft_price = get_Heston_fft_call(0.0, 0.0, params[4], r, params[3], params[1], params[0], params[2], 0.5, N=2**10, dk=1e-3)
fft_price

C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in tanh
  from ipykernel import kernelapp as app
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in cosh
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in sinh
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in cdouble_scalars


nan

In [82]:
def objective_func(x):
    res = 0.0
    for i in df_surf.index:
        T = df_surf.at[i,'tenor']
        log_k = df_surf.at[i,'log_k']
        fft_price = get_Heston_fft_call(log_k, 0.0, x[4], r, x[3], x[1], x[0], x[2], T, N=2**12, dk=1e-2)
        price = df_surf.at[i,'price']
        res += (fft_price-price)**2
    return res

In [83]:
objective_func(params)

C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in tanh
  from ipykernel import kernelapp as app
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in cdouble_scalars
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in cosh
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in sinh
C:\Users\golde\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in cdouble_scalars


nan

In [42]:
df_surf

,tenor,log_k,price,test
0,0.019178,-0.130988,0.124088,10
1,0.019178,-0.108516,0.103274,10
2,0.019178,-0.075726,0.075043,10
3,0.019178,-0.065030,0.064995,10
4,0.019178,-0.054448,0.055225,10
...,...,...,...,...
194,2.032877,0.195696,0.067188,10
195,2.032877,0.227957,0.056621,10
196,2.032877,0.304330,0.037561,10
197,2.032877,0.388887,0.028829,10
